# [Info-290] Commercial Uses of Data

## Prompt

### Background
Earlier this year, Instacart publicly released an anonymized dataset consisting of over 3 million Instacart orders with information from what was purchased and by whom (user ids) and when in the day and week purchases were made. Many retailers, including grocery stores, sell data to data brokers in order to better profile consumers. Often these purchases are linked to credit and debit cards that reveal identity. For more information, here are links [on data brokers](https://www.propublica.org/getinvolved/item/discussion-how-do-data-brokers-impact-you), [on data purchashing firms](http://www.npr.org/sections/alltechconsidered/2016/07/11/485571291/firms-are-buying-sharing-your-online-info-what-can-you-do-about-it), and [Acxiom](http://www.nytimes.com/2012/06/17/technology/acxiom-the-quiet-giant-of-consumer-database-marketing.html?pagewanted=all&_r=0&mtrref=undefined&mtrref=www.nytimes.com&mtrref=www.nytimes.com), a huge database marketer. 


### Assignment
**Using Instacart’s data, try to profile consumers and specific groups within the dataset. For a couple of options, you could try to find a particular demographic, such as millennials, or a group a health insurance company might deem as high-risk.** Included is example analysis code that starts off by pinpointing users who have ordered any kind of potato chips in more than 50% of their orders. 

You're free to use whatever tools you're comfortable with. [`pandas`](https://pandas.pydata.org/pandas-docs/stable/tutorials.html) is a great library intended for fast data analysis and manipulation of large volumes; however, in this notebook, the included example of data analysis uses Berkeley's [`datascience`](http://data8.org/datascience/) module. 

To help clarify the content of the data, the Instacart data dictionary can be found [here](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b).


### More Details
The CSVs within `instacart_data` folder can be opened with a `table.read_table` method. Examples on how to do so are below.  

Here is a brief description of some elements of the more confusing (and larger!) CSVs:

`orders_subset` - Each row represents one order or trip to the grocery store. `order_number` refers to whether the order was the user’s first or fifteenth. `order_dow` refers to the day of week the order was placed with Saturday as 0 and Sunday as 1. This CSV contains a subset of the original data. 

`order_products__prior_subset` - Each row represents one product in an order. `add_to_cart_order` refers to the order in which the product was added to the cart. 

On data integrity, `order_products__prior_subset` holds the data of the subset of users found in `orders_subset`. `orders_subset` has unaltered data for 10% of the unique users found in the original `orders` CSV file. Original data can be downloaded [here](https://www.instacart.com/datasets/grocery-shopping-2017). 

### Caveat
When analyzing the data, especially with a large dataset, be sure to *filter* as much as possible the individual dataframes and then join as needed. In addition, when checking out what each CSV looks like, it’s recommended to use the **`show`** method, rather than loading the entire table — this might crash your notebook (e.g. `table.show(5)`). Note to run terminal commands such as *head* in Jupyter notebook, attach an exclamation point as a prefix to the command (e.g. `!head file.csv`)

### To help get you going...

In [ ]:
# importing the packages that we'll need for 
# our most basic functionality
from datascience import *
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

# Import other libraries you would like to use below


Below we will use `head` to show the first couple of lines of `products.csv`.

In [ ]:
!head instacart_data/products.csv

Next we read in some of the csv's and store them as `Table` objects from the `datascience` package.

In [ ]:
# Run this cell to open and read in some data files within instacart_data
orders = Table().read_table('instacart_data/orders_subset.csv')
orders_products = Table().read_table('instacart_data/order_products__prior_subset.csv')
products = Table().read_table('instacart_data/products.csv')

To take a peek at the data stored in the table, we can use the `.show(n)` method on the table, where `n` is the number of rows that you want to display.

In [ ]:
# Taking a small look into the first four rows of products -- highly recommended to use .show for large tables 
products.show(4)

### Analysis Example
*If you're comfortable with data analysis and Python, feel free to skip this section and get right to analyzing!*

To start with, the example seeks to find the group of Instacart users who've bought potato chips consistently. In this particular context, consistently is defined as more than 50% of a user's orders have potato chips in the cart. Using this potato-chip loving crowd, the example will then try to discover more interesting characteristics shared amongst the crowd in order to further profile and target the group. 

In [ ]:
# Filtering products to just show products that fit into the category of snacks 
snacks = products.where('department_id', 19) # Why 19? How did I know that snacks are located in department 19? 
snacks

#### In addition to orders, orders_products, and products, you have access to two other datasets -- aisles and departments -- located in the folder named instacart_data. 
To open those datasets, recall how we read products into a table.

In [ ]:
"""
The following method filters the products table for products with names with specific phrases in them. 
To use the method, call the method with a case-insensitive string such as 'potato chips' and a table that contains 
at least the product_name column. The table could be products or another table filtered from products.

The method returns a table that shows products with the phrase in its name. If the table is empty, then perhaps
the phrase given was too specific. Consider trying the method again with a more general phrase. 

Examples of using the method:

apple_sauce = find_products_by_phrase('apple sauce', products)
toilet_paper = find_products_by_phrase('toilet paper', products)
"""

def find_products_by_phrase(phrase, table):
    if type(phrase) != str:
        print("Make sure that the phrase has either single or double quotes around it!")
    if 'product_name' not in table.labels:
        print("Make sure that the table has a column exactly named product_name!")
    return table.where('product_name', lambda x: True if x.lower().find(phrase) != -1 else False)

In [ ]:
potato_chips = find_products_by_phrase("potato chips", snacks)
potato_chips.show(5) # Delicious...

#### Now that we have 196 potato chips products, we're going to take a look at the users who have ordered them.
There are a couple of questions we need to answer before we can get to what we want. 
First question: How are we going to link what we found in `potato_chips` to the other tables? 
Second question: Which of the other tables should we be looking at? Which table first? 

We're going to answer the second question before the first. Because we want to have a specific group of users based on the content of their orders, we need to have a connection between orders (specifically products in each order) and users. Of the two tables that have information about orders, let's look at the column names to figure out exactly what type of information each has. 

In [ ]:
orders.show(2)

In [ ]:
orders_products.show(2)

It seems that the `potato_chips` table has `product_id` in common with the `orders_products` table. Also, the `orders_products` table shares `order_id` with the `orders` table. 

To circle back to the two questions: For the second question, we're going to look at the `orders_products` and `orders` tables in that order. For the first question, we're going to find the relevant `order_id`s in `orders_products` using the `product_id`s in table `potato_chips` and then filter `orders` to only contain those relevant `order_id`s. Let's do it! 

In [ ]:
"""
In this long line of code, using select potato_chips is made a smaller table with only the column product_id; 
similarly, orders_products is made into a table with only two columns, order_id and product_id. 
Cutting down tables (temporarily) to only contain necessary information and thenjoining decreases the 
amount of time it takes to perform the join. 

orders_products_pchips holds two columns both of which are tied to the potato chips we found in our potato_chips table.
The first column product_id refers to a specific kind of potato chip product. The second column order_id refers 
to the specific order a user placed one day. 

Note: This cell will take approximately 30 seconds to run. 
""" 

orders_products_pchips = orders_products.select(['order_id', 'product_id'])\
                                        .join('product_id', potato_chips.select('product_id'))
orders_products_pchips.show(3)

In [ ]:
"""
Because eval_set and days_since_prior_order aren't of interest to me, those two columns were dropped from orders to 
make the table smaller upon execution of join. What results is a table arbitrarily named pchips_orders_users that 
connects users to their orders containing potato chips. 
"""

pchips_orders_users = orders_products_pchips.join('order_id', orders.drop(['eval_set', 'days_since_prior_order']))
pchips_orders_users.show(3)

In [ ]:
"""
Now we want to filter pchips_orders_users to only contain the users who ordered some kind of potato chips at least
in 50% of their orders that exist in our data set. 

To get a group of users who fit the above criterion, we group pchips_orders_users to obtain the count or number of 
orders by user containing potato chips. Then using group and join between orders and itself to find the max order 
number or total number of orders placed by user. We join both of these tables to get the desired user ids. 
"""

pchips_users = pchips_orders_users.group('user_id').relabel('count', 'pchip order count')\
                                .join('user_id', orders.select(['user_id', 'order_number']).group('user_id', max))
pchips_users.show(5)

In [ ]:
# Here we add a column, pchip_freq_buyer, to the new pchips_users table which holds True if that user bought
# potato chips in more than half of his/her/their orders. 

pchips_users = pchips_users.with_column('pchip_freq_buyer', 
                            pchips_users.apply(lambda count, total: True if count/total > 0.5 else False, 
                                                                   ['pchip order count', 'order_number max']))
pchips_users.show(5)

In [ ]:
print("There are", pchips_users.where('pchip_freq_buyer', True).num_rows, 
                                                          "users who appear to be frequent buyers of potato chips!")

In [ ]:
# Filtering pchips_orders_users to only contain users in pchips_users for which pchip_freq_buyer is True

pchips_freq_buyers = pchips_orders_users.join('user_id', 
                                              pchips_users.where('pchip_freq_buyer', True).select('user_id'))
pchips_freq_buyers.show(5)

#### Next Steps
We're at the point where we have a table, pchips_freq_buyers, with which we can find all sorts of interesting characteristics about this population. At this point, hopefully you feel comfortable conducting your own analyses. If not, the following code will finish off, for now, this example. 

In [ ]:
# Question: What's the most popular kind of potato chips? 
popular_pchip_row = pchips_freq_buyers.group('product_id').sort('count', descending=True).row(0)
popular_pchip_id = popular_pchip_row[0]
print("Most popular kind among this user group is:", 
      products.where('product_id', popular_pchip_id).column('product_name')[0])

### Some visualizations of the data findings: 
![alt text](images/example_analysis_1.png)
#### Compare the above with this table of bestsellers based on the entire original dataset: 
![alt text](images/compare_1_a.png)

![alt text](images/example_analysis_2.png)
![alt text](images/example_analysis_3.png)
#### Compare the above with these bar charts based on data the entire orders_subset: 
![alt text](images/compare_2.png)
![alt text](images/compare_3.png)

*The remaining code of the example is all about the visualizations shown here. Feel free to reference the code if you would like to build a horizontal bar chart or histogram.*

In [ ]:
plt.hist(pchips_freq_buyers.column('order_dow'), color='green')
plt.title('DoW for Orders Containing Potato Chips by Frequent Buyers of Potato Chips', y=1.08)
plt.xlabel('DoW (0=Saturday, 1=Sunday)')

In [ ]:
plt.hist(pchips_freq_buyers.column('order_hour_of_day'), color='green')
plt.title('Hour of Day for Orders Containing Potato Chips by Frequent Buyers of Potato Chips', y=1.08)
plt.xlabel('Hour')

In [ ]:
# Question: What other products are ordered frequently by this specific group of people? (this cell will take a while)

freq_pchip_buyers_id = pchips_users.where('pchip_freq_buyer', True).select('user_id') 

# Getting the order ids 
freq_pchip_buyers_orders = orders.select(['user_id', 'order_id']).join('user_id', freq_pchip_buyers_id)

# Linking the order ids to product ids 
freq_pchip_buyers_productids = orders_products.select(["order_id", "product_id"])\
                                              .join('order_id', freq_pchip_buyers_orders)
    
# Linking product ids to product names
freq_pchip_buyers_products = freq_pchip_buyers_productids.select(['product_id'])\
                                                         .join('product_id', 
                                                               products.select(['product_id', 'product_name']))
     
# Getting the top 10 products most commonly purchased by this group of potato chip frequent buyers
# Note that the 'count' column refers to number of orders
freq_pchip_buyers_products = freq_pchip_buyers_products.group('product_name')\
                                                       .sort('count', descending=True)\
                                                       .take(np.arange(10))
freq_pchip_buyers_products

In [ ]:
fig, ax = plt.subplots()
ypos = np.arange(10)
ax.barh(ypos, freq_pchip_buyers_products.column('count'), color='green')
ax.set_yticklabels(freq_pchip_buyers_products.column('product_name'))
ax.set_yticks(ypos)
ax.invert_yaxis() 
ax.set_xlabel('Count')
ax.set_title('Bestsellers Amongst the Potato-Chips Frequent Buyers', y=1.08)